In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open("C:\\Users\\alackey\\.secret\\yelp_api.json") as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# set our API call parameters and filename before the first call
LOCATION = 'Edison, NJ,08817'
TERM = 'Burger'

In [4]:
JSON_FILE = f"C:\\Users\\alackey\\Data\\results_in_progress_burger.json"
JSON_FILE

'C:\\Users\\alackey\\Data\\results_in_progress_burger.json'

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] C:\Users\alackey\Data\results_in_progress_burger.json not found. Saving empty list to file.


In [6]:
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [7]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
## How many results total?
total_results = results['total']
total_results

434

In [9]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [10]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

22

In [11]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [12]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/22 [00:00<?, ?it/s]

In [13]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/22 [00:00<?, ?it/s]

In [14]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,hmrXk8jrLredlvQmcYtvjA,418-burgers-highland-park,418 Burgers,https://s3-media4.fl.yelpcdn.com/bphoto/Kyh-RB...,False,https://www.yelp.com/biz/418-burgers-highland-...,202,"[{'alias': 'tradamerican', 'title': 'American ...",4.5,"{'latitude': 40.49993610299, 'longitude': -74....","[pickup, delivery]",$,"{'address1': '418 Raritan Ave', 'address2': ''...",+17325432484,(732) 543-2484,2724.977914
1,nMU0vP0BNmc-tDtoa2u_6w,98k-hamburger-edison,98k Hamburger,https://s3-media2.fl.yelpcdn.com/bphoto/2XrCL0...,False,https://www.yelp.com/biz/98k-hamburger-edison?...,5,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",4.0,"{'latitude': 40.5151054, 'longitude': -74.4032...","[pickup, delivery]",NaN,"{'address1': '1857 Lincoln Hwy', 'address2': '...",+17322028998,(732) 202-8998,406.606325
2,IOyhF0TRcIFm5dwdIsUjVA,diesel-and-duke-new-brunswick-2,Diesel and Duke,https://s3-media2.fl.yelpcdn.com/bphoto/zxwFwg...,False,https://www.yelp.com/biz/diesel-and-duke-new-b...,207,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 40.4997241, 'longitude': -74.4532...","[pickup, delivery]",$,"{'address1': '139 Easton Ave', 'address2': Non...",+17322461001,(732) 246-1001,4865.765785
3,ao9q86jxXnHFbanIyUYbvQ,tommys-tavern-tap-edison-2,Tommy's Tavern + Tap,https://s3-media4.fl.yelpcdn.com/bphoto/5Qt7lG...,False,https://www.yelp.com/biz/tommys-tavern-tap-edi...,74,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 40.517133076968214, 'longitude': ...",[],$$,"{'address1': '1007 US-1', 'address2': '', 'add...",+17329022701,(732) 902-2701,2322.428159
4,sDFz3CorqCqxTL9_HFPRMg,white-rose-hamburgers-highland-park-2,White Rose Hamburgers,https://s3-media4.fl.yelpcdn.com/bphoto/qX4dGA...,False,https://www.yelp.com/biz/white-rose-hamburgers...,254,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.0,"{'latitude': 40.502314, 'longitude': -74.415314}","[pickup, delivery]",$,"{'address1': '154 Woodbridge Ave', 'address2':...",+17327771881,(732) 777-1881,1895.811905


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
429,wh5_74yMCk3fB4jJ5S7YOA,bagel-market-and-deli-somerset,Bagel Market and Deli,https://s3-media3.fl.yelpcdn.com/bphoto/RoICq0...,False,https://www.yelp.com/biz/bagel-market-and-deli...,65,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",3.5,"{'latitude': 40.51094, 'longitude': -74.485065}","[delivery, pickup]",$,"{'address1': '900 Easton Ave', 'address2': '',...",+17324480440,(732) 448-0440,7305.637962
430,-qTa6tVuQ2NyovjSVjGryA,vinnies-pizza-and-subs-east-brunswick,Vinnie's Pizza & Subs,https://s3-media3.fl.yelpcdn.com/bphoto/3suTCz...,False,https://www.yelp.com/biz/vinnies-pizza-and-sub...,79,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"{'latitude': 40.435424, 'longitude': -74.422942}","[delivery, pickup]",$,"{'address1': '429 Ryders Ln', 'address2': '', ...",+17322571111,(732) 257-1111,8934.226795
431,TLJtWmg_wivqREGLG6pNnA,bagel-bazaar-deli-gril-and-catering-piscataway,Bagel Bazaar Deli Gril & Catering,https://s3-media3.fl.yelpcdn.com/bphoto/kL-4s3...,False,https://www.yelp.com/biz/bagel-bazaar-deli-gri...,44,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",4.0,"{'latitude': 40.580493, 'longitude': -74.455182}","[delivery, pickup]",$,"{'address1': '474 S Washington Ave', 'address2...",+17326297800,(732) 629-7800,8824.442354
432,rB_vf8FVsyUD_-HsoYzDSA,nunzios-pizzeria-iselin,Nunzio's Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/X-GKS3...,False,https://www.yelp.com/biz/nunzios-pizzeria-isel...,75,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 40.560805, 'longitude': -74.309481}","[delivery, pickup]",$,"{'address1': '675 Rte 1 S', 'address2': None, ...",+17327500990,(732) 750-0990,9176.742974
433,jLdBrLwji5fGBBAjuxrutw,upper-crust-woodbridge-township,Upper Crust,https://s3-media1.fl.yelpcdn.com/bphoto/vG3IO5...,False,https://www.yelp.com/biz/upper-crust-woodbridg...,16,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",2.5,"{'latitude': 40.5692367553711, 'longitude': -7...",[delivery],$$,"{'address1': '37 Gill Ln', 'address2': '', 'ad...",+17325270104,(732) 527-0104,8880.522219


In [15]:
# check for duplicate IDs
final_df.duplicated(subset='id').sum()

1

In [16]:
# save the final results to a compressed csv
final_df.to_csv('C:\\Users\\alackey\\Data\\final_results_burger.csv.gz', compression='gzip',index=False)